# Grid Reference Triangles

The request is for, *"An application that, by inputting three grid references (making a triangle), can work out and display distance and bearing, as well as internal angles."* Without specifics of the use case it is difficult to know exactly what to build without making several assumptions:

- Grids will be Ordnance Survey National Grid (Format SU625323)
- Accuaracy is not too imprtant as distances are short (walking beween points not flying, etc)
- No magnetic variation and flat earth are fine

There is a good summary of the National Grid system here: https://www.ordnancesurvey.co.uk/documents/resources/guide-to-nationalgrid.pdf
Attribution https://www.movable-type.co.uk/scripts/os-grid-dist.html Chris Veness from JS version

Math required for calcualtions
Sample data for testing

In [1]:
import math as m


In [ ]:
# Test data
sample = 'NK625323'

sample_1 = 'SU625323'
sample_2 = 'ST320560'
sample_3 = 1000
sample_4 = 700
sample_5 = 800

dist_a = 8
dist_b = 6
dist_c = 7

## Setup Functions
Function to convert National Grid Reference to fully numeric 1m grid reference with 100km numeric grids (SU625323 -> 462500 132300)

In [2]:
def gridref_numeric(gridref):
    # Check gridref has 2 letters at start and even amount of digits following
    if gridref.strip()[:2].isalpha() and gridref.strip()[2:].isnumeric() and len(gridref.strip()[2:])%2 == 0:
        # Convert letters to numbers A,B,C -> 0,1,2
        letter_E = ord(gridref.upper()[0]) - ord('A'.upper())
        letter_N = ord(gridref.upper()[1]) - ord('A'.upper())
        # Letter I is not used so adjust H,J,K -> 7,8,9
        if letter_E > 7:
            letter_E -= 1
        if letter_N > 7:
            letter_N -= 1
        # Convert grid letters into 100km-square index from false origin at grid SV (SU -> 4,1)
        easting = (((letter_E+3)%5)*5) + (letter_N%5)
        northing = (19-((m.floor(letter_E/5)))*5) - (m.floor(letter_N/5))
        # Get numerical part of gridref and split easting and northing (625323 -> 625, 323)
        gridref_numbers = gridref[2:]
        gr_easting = gridref_numbers[:len(gridref_numbers)//2]
        gr_northing = gridref_numbers[len(gridref_numbers)//2:]
        # Normalise to 1m grid / 10-figure grid reference (625 323 -> 62500 32300)
        gr_easting = '{:0<5}'.format(gr_easting)
        gr_northing = '{:0<5}'.format(gr_northing)
        # Add grid letter number to front and convert str to int (4 62500, 1 32300 -> 462500 132300)
        easting = int(str(easting) + gr_easting)
        northing = int(str(northing) + gr_northing)
        return easting,northing
    else:
        return print('Grid Reference is in wrong format')

(462500, 832300)

In [ ]:
# Test the gridref_numeric() function
gridref_numeric(sample)

In [6]:
def grid_distance(grid_1, grid_2):
    # Function to calculate distance between 2 National Grid References - can be different types but must be valid (SU625323 and NU3256)

    # Get fully numeric grids
    gr_1 = gridref_numeric(grid_1)
    gr_2 = gridref_numeric(grid_2)

    # Get easting and northing distances between grid_1 and grid_2
    delta_E = gr_2[0] - gr_1[0]
    delta_N = gr_2[1] - gr_1[1]

    # Use pythagoras to get distance between points (a^2 = b^2 + c^2)
    dist = round(m.sqrt(delta_E**2 + delta_N**2))
    return dist

132635 m


In [ ]:
# Test the grid_distance() function
print(grid_distance(sample_1, sample_2),'m')

In [7]:
def grid_bearing(grid_1, grid_2):
    # Function to calculate bearing between 2 National Grid References - can be different types but must be valid (SU625323 and NU3256)

    # Get fully numeric grids
    gr_1 = gridref_numeric(grid_1)
    gr_2 = gridref_numeric(grid_2)

    # Get easting and northing distances between grid_1 and grid_2
    delta_E = gr_2[0] - gr_1[0]
    delta_N = gr_2[1] - gr_1[1]

    # Use arctan to get the bearing convert from radians to degrees
    bearing = round((90-(m.atan2(delta_N, delta_E)/m.pi*180)+360) % 360)
    return bearing

280 degrees


In [ ]:
# Test the grid_bearing() function
print(grid_bearing(sample_1, sample_2),'degrees')

In [16]:
def grid_angle(dist_a,dist_b,dist_c):
    # Function to find triangle angles between grid points (Point A has angle A. Distance a = B to C, b = A to C, c = A to B )
    angle_A = round(m.degrees(m.acos((dist_b**2 + dist_c**2 - dist_a**2) / (2 * dist_b * dist_c))),2)
    return angle_A
    


75.52 degrees


In [ ]:
# Test the grid_angle() function
print(grid_angle(dist_a, dist_b, dist_c),'degrees')

In [44]:
gridref = 'SU625323'

if gridref.strip()[:2].isalpha() and gridref.strip()[2:].isnumeric() and len(gridref.strip()[2:])%2 == 0:
    letter_E = ord(gridref.upper()[0]) - ord('A'.upper())
    letter_N = ord(gridref.upper()[1]) - ord('A'.upper())
    print(letter_E,letter_N)
    if letter_E > 7:
        letter_E -= 1
    if letter_N > 7:
        letter_N -= 1
    print('If over I',letter_E, letter_N)
    # Convert grid letters into 100km-square index from false origin (grid SV)
    easting = (((letter_E+3)%5)*5) + (letter_N%5)
    northing = (19-((m.floor(letter_E/5)))*5) - (m.floor(letter_N/5))
    print('easting =', easting)
    print('northing =', northing)
    gridref_numbers = gridref[2:]
    gr_easting = gridref_numbers[:len(gridref_numbers)//2]
    gr_northing = gridref_numbers[len(gridref_numbers)//2:]
    print(gr_easting, gr_northing)
    # Normalise to 1m grid / 10-figure grid reference
    gr_easting = '{:0<5}'.format(gr_easting)
    gr_northing = '{:0<5}'.format(gr_northing)
    print(gr_easting, gr_northing)
    easting = int(str(easting) + gr_easting)
    northing = int(str(northing) + gr_northing)
    print('easting =', easting)
    print('northing =', northing)
else:
    print('Grid Reference is in wrong format')


18 20
If over I 17 19
easting = 4
northing = 1
625 323
62500 32300
easting = 462500
northing = 132300
